<a href="https://colab.research.google.com/github/Doppler-67/sentiment_analysis/blob/main/embeddings_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

# **Data split**

In [2]:
data = pd.read_csv('/content/drive/MyDrive/sentiment_analysis/prepared_data.csv')

In [3]:
data.head()

,text,toxic,clear_text
0,Нормальный парень никогда не станет унижать бы...,True,нормальный парень никогда не станет унижать бы...
1,"нихачу в универ обратно, хочу быть эльфиком(",False,нихачу в универ обратно хочу быть эльфиком
2,@EvgenyFeldman с ним уже пару месяцев такое. н...,False,с ним уже пару месяцев такое не обращай внимание
3,"@elenabodya заебись, люблю @elenabodya , сэко...",True,заебись люблю сэкономил к на куртке
4,"лазила по ВК и наткнулась на девочку с ником ""...",True,лазила по вк и наткнулась на девочку с ником к...


Сгенерировать эмбеддинги сразу для всего датасета не получится, из-за нехватки оперативной памяти, поэтому поделим данные на батчи и будем последовательно генерировать эмбеддинги и сохранять их в csv файлы, а потом объединим их в один набор данных.

Разделим данные на 6 батчей.

In [4]:
batches = np.array_split(data, 6)

# **Embeddings generation**

In [5]:
tokenizer = transformers.BertTokenizer('/content/drive/MyDrive/sentiment_analysis/vocab.txt')

In [6]:
tokenized = batches[-1]['clear_text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

In [7]:
max_len = 69 # max tokens in texts in corpus

In [8]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [9]:
config = transformers.BertConfig.from_json_file(
    '/content/drive/MyDrive/sentiment_analysis/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/content/drive/MyDrive/sentiment_analysis/rubert_model.bin', config=config)

Some weights of the model checkpoint at /content/drive/MyDrive/sentiment_analysis/rubert_model.bin were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Для того, чтобы обработать эти 6 батчей в цикле последовательно не хватает оперативной памяти, поэтому будет по очереди запускать каждый батч.

In [10]:
batch_size = 150
embeddings = []
amount_of_batches = padded.shape[0] // batch_size
for i in notebook.tqdm(range(amount_of_batches)):
        if i == amount_of_batches-1: # last batch
            batch = torch.LongTensor(padded[batch_size*i:]) 
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:])
        else:
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/240 [00:00<?, ?it/s]

Соединим все эмбеддинги.

In [11]:
embeddings = np.concatenate(embeddings)

# **Saving embeddings**

Сохраним эмбеддинги вместе с таргетом в csv файл.

In [12]:
embeddings = pd.DataFrame(embeddings)

In [13]:
embeddings['toxic'] = batches[-1]['toxic']

In [15]:
embeddings.to_csv('emb_5.csv', index=False)